## Importing Libraries

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error,r2_score
%matplotlib inline
from math import asin
from datetime import datetime, timedelta
import swisseph as swe
from keras.models import Sequential
from keras.layers import Dense, Activation,LSTM,Dropout,LeakyReLU,Input,TimeDistributed,GRU
from keras.layers.wrappers import Bidirectional
from keras.optimizers import Adam
import keras
from tqdm import tqdm_notebook,tqdm,trange
import math


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


# Using FXCM to get more data

## Reading Datasets and calculating Coordinates

In [2]:
eur=pd.read_csv('EURUSD.csv',parse_dates=['Date'])
#Change name of dataset, whichever dataset you wanna use


In [3]:
dates = [date_time.to_pydatetime() for date_time in eur["Date"]] 
#this are the dates that we'll use for calculating coordinates
eur['Year']=eur['Date'].map(lambda x:x.year)
eur['Month']=eur['Date'].map(lambda x:x.month)


## Change date as you want

In [4]:
from dateutil import rrule, parser
# date1 = '2018-12-01'
# date2 = '2019-1-28'
date2 = '2019-12-28'
date1 = '2019-01-28'
datesx = list(rrule.rrule(rrule.DAILY, dtstart=parser.parse(date1), until=parser.parse(date2)))

## The function for calculating coordinates

In [5]:
'''Just put dates in the function, in the format(to_pydatetime, function will take care of the rest'''#planet order for columns in csv
# p = '2314567'    
EARTH_RADIUS=6378136.6
AUNIT=1.49597870691e+11       #au in meters, AA 2006 K6
DEGTORAD=0.0174532925199433

planet_idx = {
                
            
                "Mercury" : 2,
                "Venus"   : 3,
                "Moon"    : 1,
                "Mars"    : 4,
                "Jupiter" : 5,
                "Saturn"  : 6,
                "Uranus"  : 7, 
                "Neptune" : 8,
                "Earth"   : 14,
            
            }
               
    
def calc_utc_coord(dates, input_planets_data):
#FLG_HELCTR works awesome so far
    def _swisseph(t, ipl):
        t = t[0]
        jd = swe.julday(t.year,t.month,t.day,t.hour+int(t.minute/60))
        rslt = swe.calc_ut(jd, ipl , swe.FLG_HELCTR | swe.FLG_EQUATORIAL | swe.FLG_SPEED)

        if ipl == 1: #Moon
            rm = swe.calc_ut(jd, ipl , swe.FLG_EQUATORIAL | swe.FLG_SPEED |swe.FLG_TRUEPOS | swe.FLG_RADIANS)
            sinhp = EARTH_RADIUS / rm[2] / AUNIT
            moondist=asin(sinhp) / DEGTORAD *3600
            return rslt[0], rslt[1], moondist

        return rslt[0], rslt[1], rslt[2]

    column_names = ['Dat']

    column_names.extend([x for x in planet_idx if x in input_planets_data])

   

    extended_dates = dates.copy()

    final_df = pd.DataFrame(index=extended_dates)
    extended_dates = np.vstack(np.asarray(extended_dates))

    for planet in column_names[1:]:

        result = np.apply_along_axis( _swisseph, 1, extended_dates, planet_idx[planet]) #Apply Swisseph func on every rows

        if "right asc." in input_planets_data[planet]:
            final_df.loc[:, planet+'_'+"right asc."] = result[:, 0] 

        if "declination" in input_planets_data[planet]:
            final_df.loc[:, planet+'_'+"declination"] = result[:, 1]

        if "distance" in input_planets_data[planet]:
            final_df.loc[:, planet+'_'+"distance"] = result[:, 2]

    return final_df.loc[dates, :]



planet_parameters = {
        "Mercury" : ["right asc.", "declination", "distance"],
        "Venus"   : ["right asc.", "declination", "distance"],
        "Moon"    : ["right asc.", "declination", "distance"],
        "Mars"    : ["right asc.", "declination", "distance"],
        "Jupiter" : ["right asc.", "declination", "distance"],
        "Saturn"  : ["right asc.", "declination", "distance"],
        "Uranus"  : ["right asc.", "declination", "distance"],
        "Earth"   : ["right asc.", "declination", "distance"],
        "Neptune" : ["right asc.", "declination", "distance"]
}



result=pd.DataFrame(calc_utc_coord(dates ,planet_parameters))
myresult=pd.DataFrame(calc_utc_coord(datesx ,planet_parameters))



In [6]:
result.reset_index(inplace=True)
myresult.reset_index(inplace=True)

In [7]:
myresult['Year']=myresult['index'].map(lambda x:x.year)
myresult['Month']=myresult['index'].map(lambda x:x.month)
myresult.drop(['index'],axis=1,inplace=True)

In [8]:
final_data=pd.concat([eur,result],axis=1)
final_data.head()

,Date,Open,High,Low,Close,Year,Month,index,Jupiter_right asc.,Jupiter_declination,...,Mars_distance,Venus_right asc.,Venus_declination,Venus_distance,Earth_right asc.,Earth_declination,Earth_distance,Neptune_right asc.,Neptune_declination,Neptune_distance
0,1993-05-11,1.2412,1.2412,1.2376,1.2376,1993,5,1993-05-11,192.404589,-3.906049,...,1.664916,253.697000,-22.494622,0.726029,227.876087,-17.826018,1.009988,290.884441,-21.371923,30.185435
1,1993-05-12,1.2376,1.2376,1.2353,1.2353,1993,5,1993-05-12,192.474359,-3.935484,...,1.664761,255.396079,-22.756455,0.726142,228.855142,-18.081259,1.010220,290.890827,-21.371245,30.185416
2,1993-05-13,1.2353,1.2353,1.2225,1.2225,1993,5,1993-05-13,192.544123,-3.964909,...,1.664597,257.100916,-23.000176,0.726253,229.836648,-18.331450,1.010450,290.897200,-21.370570,30.185398
3,1993-05-14,1.2225,1.2345,1.2225,1.2345,1993,5,1993-05-14,192.613882,-3.994323,...,1.664423,258.811157,-23.225521,0.726363,230.820603,-18.576508,1.010676,290.903562,-21.369894,30.185379
4,1993-05-18,1.2264,1.2264,1.2203,1.2203,1993,5,1993-05-18,192.892929,-4.111896,...,1.663629,265.698237,-23.938524,0.726774,234.780751,-19.503820,1.011546,290.928962,-21.367163,30.185303


In [9]:
final_data.drop(['Open','High','Low','index'],axis=1,inplace=True)

## Feature Engineering and Data Cleaning

## Now we're making features

In [10]:
features=final_data[['Month','Year',
       'Moon_right asc.', 'Moon_declination', 'Moon_distance',
       'Mercury_right asc.', 'Mercury_declination', 'Mercury_distance',
       'Venus_right asc.', 'Venus_declination', 'Venus_distance',
       'Mars_right asc.', 'Mars_declination', 'Mars_distance',
       'Jupiter_right asc.', 'Jupiter_declination', 'Jupiter_distance',
       'Saturn_right asc.', 'Saturn_declination', 'Saturn_distance',
       'Uranus_right asc.', 'Uranus_declination', 'Uranus_distance',
       'Neptune_right asc.', 'Neptune_declination', 'Neptune_distance',
       'Earth_right asc.', 'Earth_declination', 'Earth_distance']]

target=final_data['Close']

In [11]:
close=np.zeros_like(final_data['Close'])
close.shape

(6311,)

In [12]:
price=[]

In [13]:
plt.figure(figsize=(30,10))
# plt.plot(hund['Close'])

# plt.figure(figsize=(30,10))
plt.plot(final_data['Close'].rolling(window=10).mean())


# As we can see, our training set comprises of all the years except 2018. All our testing is done on 2018, which is an unseen year for our model, still it performs very well

In [14]:
columns=['Month','Year',
       'Moon_right asc.', 'Moon_declination', 'Moon_distance',
       'Mercury_right asc.', 'Mercury_declination', 'Mercury_distance',
       'Venus_right asc.', 'Venus_declination', 'Venus_distance',
       'Mars_right asc.', 'Mars_declination', 'Mars_distance',
       'Jupiter_right asc.', 'Jupiter_declination', 'Jupiter_distance',
       'Saturn_right asc.', 'Saturn_declination', 'Saturn_distance',
       'Uranus_right asc.', 'Uranus_declination', 'Uranus_distance',
       'Neptune_right asc.', 'Neptune_declination', 'Neptune_distance',
       'Earth_right asc.', 'Earth_declination', 'Earth_distance']

new_columns=['new'+i for i in columns]

In [15]:
def normalize(*df):
    dfs=list(df)
    min_max_scaler =MinMaxScaler()
    column=['Month','Year',
       'Moon_right asc.', 'Moon_declination', 'Moon_distance',
       'Mercury_right asc.', 'Mercury_declination', 'Mercury_distance',
       'Venus_right asc.', 'Venus_declination', 'Venus_distance',
       'Mars_right asc.', 'Mars_declination', 'Mars_distance',
       'Jupiter_right asc.', 'Jupiter_declination', 'Jupiter_distance',
       'Saturn_right asc.', 'Saturn_declination', 'Saturn_distance',
       'Uranus_right asc.', 'Uranus_declination', 'Uranus_distance',
       'Neptune_right asc.', 'Neptune_declination', 'Neptune_distance',
       'Earth_right asc.', 'Earth_declination', 'Earth_distance']
    
    for i in column:
        for j in range(len(dfs)):      
            if j==0:
                dfs[j]['new'+i] = min_max_scaler.fit_transform(np.array(dfs[j][i]).reshape(-1,1))
            else:
                dfs[j]['new'+i] = min_max_scaler.transform(np.array(dfs[j][i]).reshape(-1,1))
    return dfs


In [16]:
x=normalize(features,myresult)
features=x[0]
myresult=x[1]

/usr/local/lib/python3.5/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype int64 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [17]:
predict=pd.concat([features[new_columns],myresult[new_columns]],axis=0)
predict.reset_index(inplace=True)
predict.drop(['index'],axis=1,inplace=True)
# for_vis=pd.concat([features[columns],myresult[columns]],axis=0)
# for_vis[:-13].to_csv('predict.csv')

# Place below the number of In-Sample examples

In [18]:
total_no_of_examples_with_close_price = features.shape[0]
total_no_of_examples_with_close_price

6311

In [19]:
train_percent = 50 #Mention the percent of data you wanna use for training/''
train_percent = train_percent/100.0

In [20]:
no_of_sample_you_wanna_train_with = math.floor(train_percent * total_no_of_examples_with_close_price)
print("So the number of examples used for training algorithm will be: ", no_of_sample_you_wanna_train_with)

trainx=features[new_columns][0:no_of_sample_you_wanna_train_with]
trainy=target[0:no_of_sample_you_wanna_train_with]
testx=features[new_columns][no_of_sample_you_wanna_train_with:] #All the remaining examples will be used as test 
testy=target[no_of_sample_you_wanna_train_with:]

So the number of examples used for training algorithm will be:  3155


In [21]:
features=np.array(features[new_columns])
predict=np.array(predict)
trainx=np.array(trainx)
trainy=np.array(trainy)
testx=np.array(testx)
testy=np.array(testy)
target=np.array(target)

In [22]:
timesteps=30
no_features=29

def modify_output(x):
    net_shape=x.size/no_features
    floor_shape=math.floor(net_shape/timesteps)
    tobe=floor_shape*timesteps
    net_shape=net_shape-tobe
    return x[0:-int(net_shape)].reshape(-1,timesteps,no_features),int(net_shape)

In [23]:
trainx,tr=modify_output(trainx)
trainy=trainy[0: -tr].reshape((trainx.shape[0],-1))
testx,te=modify_output(testx)
testy=testy[0: -te].reshape((testx.shape[0],-1))
features,fe=modify_output(features)
target=target[0: -fe].reshape((features.shape[0],-1))
predict,pr=modify_output(predict)

## Using LSTM Model

Best combination so far is of 5 units in first layer and four units in second layer.

In [24]:
model=Sequential()
model.add(LSTM(units=15,input_shape=(timesteps,no_features),return_sequences=True))
model.add(LSTM(10,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(trainx.shape[1])) 
ad = Adam(lr=0.01, clipnorm=1)
             
model.compile(loss="mae", optimizer=ad) 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 30, 15)            2700      
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                1040      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 30)                330       
Total params: 4,070
Trainable params: 4,070
Non-trainable params: 0
_________________________________________________________________


In [25]:
model.load_weights('model.h5')
history = model.fit(trainx, trainy, epochs=1,batch_size=1000, validation_data=(testx, testy), verbose=1, shuffle=False)

#If you wanna train, change epoch from 1 to 500. Remember, also more the number of epoch, more you train, so chances are that you'll overfit, so try for 300-500 epoch. 
#Sometimes you might have to train for even 1000 epoch, depending on data.

Train on 105 samples, validate on 105 samples
Epoch 1/1
105/105 [==============================] - 2s 20ms/step - loss: 0.0420 - val_loss: 0.1054


In [26]:
#DO NOT CHANGE PARAMETERS OF THIS CELL

inputt=Input(shape=(timesteps,no_features))
bi_one=Bidirectional(LSTM(10, return_sequences=True,activation='linear'))(inputt)
bi_sec=Bidirectional(LSTM(15, return_sequences=False,activation='linear'))(bi_one)
output=Dense(trainx.shape[1], activation='linear')(bi_sec)             

model1 = keras.models.Model(inputt,output)

ad = Adam(lr=0.01, clipnorm=1)

             
model1.compile(loss="mae", optimizer=ad) 
model1.summary()

history = model1.fit(features, target, epochs=1,batch_size=1000, validation_data=(testx, testy), verbose=1, shuffle=False)
model1.load_weights('model1.h5')


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 30, 29)            0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 30, 20)            3200      
_________________________________________________________________
bidirectional_2 (Bidirection (None, 30)                4320      
_________________________________________________________________
dense_2 (Dense)              (None, 30)                930       
Total params: 8,450
Trainable params: 8,450
Non-trainable params: 0
_________________________________________________________________
Train on 210 samples, validate on 105 samples
Epoch 1/1
210/210 [==============================] - 2s 10ms/step - loss: 1.2550 - val_loss: 1.1038


In [33]:
ans=model.predict(predict)
ans=ans.reshape((-1,1))
ans1=model1.predict(predict)
ans1=ans1.reshape((-1,1))

beta=0.9


for i in range(ans.shape[0]):
    
    if i==0:
        ans[i]=beta*target[-1][-1]+(1-beta)*ans[i]
    else:
        ans[i]=beta*ans[i-1]+(1-beta)*(0.85 *ans[i] + 0.2*ans1[i])
        

ans=pd.DataFrame(ans,columns=['hel'])
plt.figure(figsize=(50,15))
plt.plot(ans['hel'],color='red',label='Prediction')
plt.axvline(no_of_sample_you_wanna_train_with,color='blue')

plt.plot(target.reshape((-1,1)),color='blue',label='Actual Value')
plt.legend( prop={'size': 20})

# If you want to save weight, remove # before model.save, it'll save weight, NOTE: It'll overwrite previous weight, so make a copy 

In [34]:
#model.save_weights('model.h5')


In [29]:
# ans=ans.reshape((-1,1))
ans=pd.DataFrame(ans)

In [30]:
ans.to_csv('new_close.csv')

In [31]:
pd.DataFrame(testy.reshape((-1,1))).describe()

,0
count,3150.000000
mean,1.286542
std,0.128522
min,1.038590
25%,1.171893
50%,1.305130
75%,1.369217
max,1.599010
